***Load Charmader Utilities***

In [1]:
import org.att.charmander.CharmanderUtils

&nbsp;

***See what tasks are currently running and pick first test***

In [8]:
val runningSimulators = CharmanderUtils.getMeteredTaskNamesFromRedis
val simulatorToInvestigate = runningSimulators(0)
runningSimulators

List(lookbusy80)

&nbsp;

***Retrieve memory-usage datapoints for the first simulator***

In [3]:
val memoryUsage= CharmanderUtils.getRDDForTask(sc, simulatorToInvestigate, "memory_usage", 100)

&nbsp;

***Just for curiosity lets look at the first data point***

In [4]:
memoryUsage.first

List(1425192395000, 4050001, 84971520)

&nbsp;

***Translate Datapoints in to a Spark-SQL Database***

In [5]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext._

case class MemoryUsage(timestamp: BigDecimal, memory: BigDecimal)

val memoryusageRdd = memoryUsage.map(p => MemoryUsage(BigDecimal(p(0).asInstanceOf[BigInt]), BigDecimal(p(2).asInstanceOf[BigInt])))
memoryusageRdd.registerTempTable("memoryusage")


&nbsp;

***Get max-memory usage***

In [9]:
val newestMax = sqlContext.sql("select max(memory) from memoryusage").first()
newestMax

[84971520]

&nbsp;

***Set the new max value in our task intelligence database***

In [7]:
CharmanderUtils.setTaskIntelligence(simulatorToInvestigate, "mem", newestMax(0).toString)

***To Verify that the value is set***

Visit [redis](http://172.31.2.11:31610/) and look for the task-intelligence section
